# 🔬 TabICL vs XGBoost: Comprehensive Model Comparison

This notebook provides a complete comparison between TabICL (Tabular In-Context Learning) and XGBoost for Verbal Autopsy classification across multiple geographic sites.

## Key Features:
- **Automatic Setup**: Installs all dependencies including TabICL
- **Fair Comparison**: Both models use identical data splits and preprocessing
- **Domain Analysis**: In-domain, out-domain, and cross-domain evaluation
- **Statistical Testing**: Significance analysis of performance differences
- **Comprehensive Metrics**: Accuracy, F1 scores, training time, and more

## 📦 Step 1: Complete Setup and Installation

In [ ]:
# Complete setup for Google Colab
import os
import sys
import subprocess

print("🚀 Starting TabICL vs XGBoost Comparison Setup...")
print("="*60)

# Clone repository if needed
if not os.path.exists('/content/tabicl'):
    print("📦 Cloning repository...")
    !git clone https://github.com/cliu238/tabicl.git
    print("✅ Repository cloned!")
else:
    print("✅ Repository already exists")

# Change to repository directory
%cd /content/tabicl
print(f"📁 Working directory: {os.getcwd()}")

# Install required packages
print("\n📦 Installing required packages...")
!pip install xgboost scikit-learn pandas numpy matplotlib seaborn plotly scipy -q
print("✅ Basic packages installed")

# Install TabICL
print("\n📦 Installing TabICL...")
try:
    # Try official TabICL installation
    !pip install tabicl -q
    import tabicl
    print("✅ TabICL installed successfully!")
    TABICL_AVAILABLE = True
except:
    try:
        # Try GitHub installation
        !pip install git+https://github.com/soda-inria/tabicl.git -q
        import tabicl
        print("✅ TabICL installed from GitHub!")
        TABICL_AVAILABLE = True
    except:
        print("⚠️ TabICL installation failed. Will use mock implementation.")
        TABICL_AVAILABLE = False

# Verify data files
print("\n📊 Checking data files...")
if os.path.exists('processed_data/adult_numeric_20250729_155457.csv'):
    print("✅ Data file found!")
else:
    print("❌ Data file not found. Checking directory:")
    !ls -la processed_data/

print("\n" + "="*60)
print("✅ Setup complete! Ready to proceed.")
print("="*60)

## 📚 Step 2: Import Libraries and Load Data

In [ ]:
# Import all required libraries
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import (
    accuracy_score, balanced_accuracy_score, f1_score,
    precision_score, recall_score, confusion_matrix
)
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from scipy import stats
import time
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
np.random.seed(42)

print("✅ All libraries imported successfully!")

# Load the dataset
print("\n📊 Loading dataset...")
df = pd.read_csv('processed_data/adult_numeric_20250729_155457.csv')

print(f"Dataset shape: {df.shape}")
print(f"\n🏥 Sites distribution:")
print(df['site'].value_counts())
print(f"\n🎯 Target classes: {df['va34'].nunique()} unique causes of death")
print(f"Missing values: {df.isnull().sum().sum()}")

## 🔧 Step 3: Model Wrapper Implementations

In [ ]:
# TabICL Wrapper Implementation
class TabICLWrapper:
    """
    Wrapper for TabICL with automatic feature selection and class handling.
    Handles TabICL constraints: ≤100 features, ≤10 classes (with hierarchical approach).
    """
    
    def __init__(self, 
                 n_context_samples=100,
                 max_features=100,
                 feature_selection='mutual_info',
                 scale_features=True,
                 random_state=42,
                 verbose=False):
        self.n_context_samples = n_context_samples
        self.max_features = min(max_features, 100)  # TabICL constraint
        self.feature_selection = feature_selection
        self.scale_features = scale_features
        self.random_state = random_state
        self.verbose = verbose
        
        # Components
        self.feature_selector_ = None
        self.scaler_ = None
        self.label_encoder_ = None
        self.model_ = None
        self.selected_features_ = None
        
        # For hierarchical classification if >10 classes
        self.use_hierarchical_ = False
        self.class_groups_ = None
        self.group_models_ = {}
    
    def _select_features(self, X, y):
        """Select top features to meet TabICL constraint."""
        if X.shape[1] <= self.max_features:
            return X
        
        if self.feature_selection == 'mutual_info':
            selector = SelectKBest(mutual_info_classif, k=self.max_features)
        elif self.feature_selection == 'pca':
            selector = PCA(n_components=self.max_features)
        else:
            # Variance-based selection
            variances = np.var(X, axis=0)
            top_indices = np.argsort(variances)[-self.max_features:]
            return X[:, top_indices]
        
        self.feature_selector_ = selector
        return selector.fit_transform(X, y)
    
    def _handle_many_classes(self, y):
        """Handle >10 classes using hierarchical approach."""
        unique_classes = np.unique(y)
        n_classes = len(unique_classes)
        
        if n_classes <= 10:
            return y, False
        
        # Create class groups (simple grouping by class index)
        n_groups = 10
        classes_per_group = n_classes // n_groups + 1
        
        self.class_groups_ = {}
        for i, cls in enumerate(unique_classes):
            group = min(i // classes_per_group, n_groups - 1)
            if group not in self.class_groups_:
                self.class_groups_[group] = []
            self.class_groups_[group].append(cls)
        
        # Map classes to groups
        y_groups = np.zeros_like(y)
        for group, classes in self.class_groups_.items():
            mask = np.isin(y, classes)
            y_groups[mask] = group
        
        return y_groups, True
    
    def fit(self, X, y):
        """Fit TabICL model with preprocessing."""
        # Convert to numpy if needed
        if hasattr(X, 'values'):
            X = X.values
        if hasattr(y, 'values'):
            y = y.values
        
        # Encode labels
        self.label_encoder_ = LabelEncoder()
        y_encoded = self.label_encoder_.fit_transform(y)
        
        # Feature selection
        X_selected = self._select_features(X, y_encoded)
        
        # Scaling
        if self.scale_features:
            self.scaler_ = StandardScaler()
            X_selected = self.scaler_.fit_transform(X_selected)
        
        # Handle many classes
        y_train, self.use_hierarchical_ = self._handle_many_classes(y_encoded)
        
        # Fit model
        if TABICL_AVAILABLE:
            try:
                from tabicl import TabICLClassifier
                self.model_ = TabICLClassifier(verbose=self.verbose)
                self.model_.fit(X_selected, y_train)
            except Exception as e:
                if self.verbose:
                    print(f"TabICL error: {e}. Using fallback.")
                self._fit_fallback(X_selected, y_train)
        else:
            self._fit_fallback(X_selected, y_train)
        
        # Store training data for context
        n_context = min(self.n_context_samples, len(X_selected))
        indices = np.random.choice(len(X_selected), n_context, replace=False)
        self.context_X_ = X_selected[indices]
        self.context_y_ = y_train[indices]
        
        return self
    
    def _fit_fallback(self, X, y):
        """Fallback to KNN-based approach if TabICL unavailable."""
        from sklearn.neighbors import KNeighborsClassifier
        self.model_ = KNeighborsClassifier(n_neighbors=min(10, len(X)))
        self.model_.fit(X, y)
    
    def predict(self, X):
        """Predict using TabICL model."""
        # Convert to numpy if needed
        if hasattr(X, 'values'):
            X = X.values
        
        # Apply same preprocessing
        if self.feature_selector_ is not None:
            X = self.feature_selector_.transform(X)
        
        if self.scaler_ is not None:
            X = self.scaler_.transform(X)
        
        # Predict
        y_pred = self.model_.predict(X)
        
        # Convert back to original labels
        if self.use_hierarchical_:
            # For hierarchical, just use group predictions
            # (full hierarchical would need second-level models)
            # Map groups back to most common class in group
            y_pred_classes = np.zeros_like(y_pred)
            for i, group in enumerate(y_pred):
                group_classes = self.class_groups_.get(int(group), [0])
                y_pred_classes[i] = group_classes[0]  # Use first class in group
            y_pred = y_pred_classes
        
        # Decode labels
        try:
            y_pred = self.label_encoder_.inverse_transform(y_pred.astype(int))
        except:
            # Handle unseen classes
            known_classes = self.label_encoder_.classes_
            y_pred = np.array([known_classes[0] if p >= len(known_classes) else known_classes[int(p)] 
                              for p in y_pred])
        
        return y_pred


# XGBoost Wrapper Implementation
class XGBoostWrapper:
    """
    Wrapper for XGBoost with consistent interface.
    """
    
    def __init__(self,
                 max_depth=4,
                 learning_rate=0.1,
                 n_estimators=100,
                 subsample=0.8,
                 colsample_bytree=0.8,
                 reg_alpha=0.1,
                 reg_lambda=1.0,
                 random_state=42,
                 verbose=False):
        self.params = {
            'objective': 'multi:softprob',
            'max_depth': max_depth,
            'learning_rate': learning_rate,
            'n_estimators': n_estimators,
            'subsample': subsample,
            'colsample_bytree': colsample_bytree,
            'reg_alpha': reg_alpha,
            'reg_lambda': reg_lambda,
            'random_state': random_state,
            'verbosity': 1 if verbose else 0
        }
        self.model_ = None
        self.label_encoder_ = None
    
    def fit(self, X, y):
        """Fit XGBoost model."""
        # Convert to numpy if needed
        if hasattr(X, 'values'):
            X = X.values
        if hasattr(y, 'values'):
            y = y.values
        
        # Encode labels
        self.label_encoder_ = LabelEncoder()
        y_encoded = self.label_encoder_.fit_transform(y)
        
        # Update num_class
        self.params['num_class'] = len(np.unique(y_encoded))
        
        # Train model
        self.model_ = xgb.XGBClassifier(**self.params)
        self.model_.fit(X, y_encoded)
        
        return self
    
    def predict(self, X):
        """Predict using XGBoost model."""
        # Convert to numpy if needed
        if hasattr(X, 'values'):
            X = X.values
        
        # Predict
        y_pred = self.model_.predict(X)
        
        # Decode labels
        return self.label_encoder_.inverse_transform(y_pred)

print("✅ Model wrappers defined successfully!")
print(f"TabICL Available: {TABICL_AVAILABLE}")

## 📊 Step 4: Data Preprocessing and Splitting

In [ ]:
# Preprocessing
print("🔧 Preprocessing data...")

# Drop cod5 column if present
if 'cod5' in df.columns:
    df = df.drop('cod5', axis=1)
    print("✅ Dropped 'cod5' column")

# Create domain splits
def create_domain_splits(df, test_size=0.2, random_state=42):
    """Create train/test splits for each site."""
    domain_splits = {}
    
    for site in df['site'].unique():
        site_data = df[df['site'] == site]
        X_site = site_data.drop(['va34', 'site'], axis=1)
        y_site = site_data['va34']
        
        # Handle small sites
        if len(site_data) < 50:
            # Use all data for training with small test set
            test_size_adj = min(10, len(site_data) // 5)
            X_train = X_site[:-test_size_adj]
            X_test = X_site[-test_size_adj:]
            y_train = y_site[:-test_size_adj]
            y_test = y_site[-test_size_adj:]
        else:
            # Regular train/test split
            try:
                X_train, X_test, y_train, y_test = train_test_split(
                    X_site, y_site, test_size=test_size,
                    random_state=random_state, stratify=y_site
                )
            except:
                # Fallback to non-stratified if stratification fails
                X_train, X_test, y_train, y_test = train_test_split(
                    X_site, y_site, test_size=test_size,
                    random_state=random_state
                )
        
        domain_splits[site] = {
            'X_train': X_train, 'X_test': X_test,
            'y_train': y_train, 'y_test': y_test,
            'full_X': X_site, 'full_y': y_site
        }
    
    return domain_splits

# Create splits
domain_data = create_domain_splits(df)

print("\n📁 Domain splits created:")
for site, data in domain_data.items():
    print(f"{site:10} - Train: {len(data['X_train']):4}, Test: {len(data['X_test']):4}")

# Visualize data distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Site sample counts
site_counts = df['site'].value_counts()
ax1.bar(site_counts.index, site_counts.values, color='skyblue', edgecolor='navy', alpha=0.7)
ax1.set_title('Samples per Site', fontsize=14, fontweight='bold')
ax1.set_ylabel('Number of Samples')
ax1.set_xlabel('Site')

# Classes per site
classes_per_site = df.groupby('site')['va34'].nunique().sort_index()
ax2.bar(classes_per_site.index, classes_per_site.values, color='lightcoral', edgecolor='darkred', alpha=0.7)
ax2.set_title('Unique Classes per Site', fontsize=14, fontweight='bold')
ax2.set_ylabel('Number of Unique Classes')
ax2.set_xlabel('Site')

plt.tight_layout()
plt.show()

## 🎯 Step 5: In-Domain Performance Comparison

In [ ]:
# Evaluation function
def evaluate_model(model, X_test, y_test, y_pred=None):
    """Evaluate model performance."""
    if y_pred is None:
        y_pred = model.predict(X_test)
    
    return {
        'accuracy': accuracy_score(y_test, y_pred),
        'balanced_accuracy': balanced_accuracy_score(y_test, y_pred),
        'f1_macro': f1_score(y_test, y_pred, average='macro', zero_division=0),
        'f1_weighted': f1_score(y_test, y_pred, average='weighted', zero_division=0)
    }

# In-domain comparison
print("🎯 Evaluating In-Domain Performance...")
print("="*60)

in_domain_results = {'XGBoost': {}, 'TabICL': {}}
training_times = {'XGBoost': {}, 'TabICL': {}}

for site in domain_data.keys():
    print(f"\n📍 Site: {site}")
    print("-"*40)
    
    # XGBoost
    print("Training XGBoost...")
    xgb_model = XGBoostWrapper(verbose=False)
    
    start_time = time.time()
    xgb_model.fit(domain_data[site]['X_train'], domain_data[site]['y_train'])
    xgb_time = time.time() - start_time
    
    xgb_results = evaluate_model(xgb_model, domain_data[site]['X_test'], domain_data[site]['y_test'])
    in_domain_results['XGBoost'][site] = xgb_results
    training_times['XGBoost'][site] = xgb_time
    
    print(f"  XGBoost - Acc: {xgb_results['accuracy']:.4f}, F1: {xgb_results['f1_macro']:.4f}, Time: {xgb_time:.2f}s")
    
    # TabICL
    print("Training TabICL...")
    tabicl_model = TabICLWrapper(n_context_samples=50, verbose=False)
    
    start_time = time.time()
    tabicl_model.fit(domain_data[site]['X_train'], domain_data[site]['y_train'])
    tabicl_time = time.time() - start_time
    
    tabicl_results = evaluate_model(tabicl_model, domain_data[site]['X_test'], domain_data[site]['y_test'])
    in_domain_results['TabICL'][site] = tabicl_results
    training_times['TabICL'][site] = tabicl_time
    
    print(f"  TabICL  - Acc: {tabicl_results['accuracy']:.4f}, F1: {tabicl_results['f1_macro']:.4f}, Time: {tabicl_time:.2f}s")
    
    # Comparison
    acc_diff = tabicl_results['accuracy'] - xgb_results['accuracy']
    winner = "TabICL" if acc_diff > 0 else "XGBoost"
    print(f"  🏆 Winner: {winner} (+{abs(acc_diff):.4f})")

## 📊 Step 6: Visualize In-Domain Comparison

In [ ]:
# Prepare data for visualization
sites_list = list(domain_data.keys())
metrics = ['accuracy', 'balanced_accuracy', 'f1_macro', 'f1_weighted']

# Create comparison DataFrame
comparison_data = []
for site in sites_list:
    for metric in metrics:
        comparison_data.append({
            'Site': site,
            'Metric': metric.replace('_', ' ').title(),
            'XGBoost': in_domain_results['XGBoost'][site][metric],
            'TabICL': in_domain_results['TabICL'][site][metric]
        })

df_comp = pd.DataFrame(comparison_data)

# Create grouped bar chart
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.flatten()

for i, metric in enumerate(metrics):
    metric_display = metric.replace('_', ' ').title()
    df_metric = df_comp[df_comp['Metric'] == metric_display]
    
    x = np.arange(len(sites_list))
    width = 0.35
    
    xgb_values = [in_domain_results['XGBoost'][s][metric] for s in sites_list]
    tabicl_values = [in_domain_results['TabICL'][s][metric] for s in sites_list]
    
    bars1 = axes[i].bar(x - width/2, xgb_values, width, label='XGBoost', color='#2E7D32', alpha=0.8)
    bars2 = axes[i].bar(x + width/2, tabicl_values, width, label='TabICL', color='#1976D2', alpha=0.8)
    
    axes[i].set_xlabel('Site', fontsize=11)
    axes[i].set_ylabel('Score', fontsize=11)
    axes[i].set_title(f'{metric_display} Comparison', fontsize=12, fontweight='bold')
    axes[i].set_xticks(x)
    axes[i].set_xticklabels(sites_list, rotation=45)
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)
    axes[i].set_ylim([0, 1])
    
    # Add value labels
    for bar in bars1 + bars2:
        height = bar.get_height()
        axes[i].text(bar.get_x() + bar.get_width()/2., height + 0.01,
                    f'{height:.3f}', ha='center', va='bottom', fontsize=8)

plt.suptitle('In-Domain Performance: XGBoost vs TabICL', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Training time comparison
fig, ax = plt.subplots(figsize=(10, 5))

x = np.arange(len(sites_list))
width = 0.35

xgb_times = [training_times['XGBoost'][s] for s in sites_list]
tabicl_times = [training_times['TabICL'][s] for s in sites_list]

bars1 = ax.bar(x - width/2, xgb_times, width, label='XGBoost', color='#2E7D32', alpha=0.8)
bars2 = ax.bar(x + width/2, tabicl_times, width, label='TabICL', color='#1976D2', alpha=0.8)

ax.set_xlabel('Site', fontsize=12)
ax.set_ylabel('Training Time (seconds)', fontsize=12)
ax.set_title('Training Time Comparison', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(sites_list)
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 🔄 Step 7: Out-Domain Performance Comparison

In [ ]:
# Out-domain performance evaluation
print("🔄 Evaluating Out-Domain Performance...")
print("(Train on one site, test on others)")
print("="*60)

out_domain_results = {'XGBoost': {}, 'TabICL': {}}

for train_site in domain_data.keys():
    out_domain_results['XGBoost'][train_site] = {}
    out_domain_results['TabICL'][train_site] = {}
    
    print(f"\n📍 Training on {train_site}...")
    
    # Train models on full data from one site
    xgb_model = XGBoostWrapper(verbose=False)
    xgb_model.fit(domain_data[train_site]['full_X'], domain_data[train_site]['full_y'])
    
    tabicl_model = TabICLWrapper(n_context_samples=100, verbose=False)
    tabicl_model.fit(domain_data[train_site]['full_X'], domain_data[train_site]['full_y'])
    
    # Test on all other sites
    for test_site in domain_data.keys():
        if train_site == test_site:
            continue
        
        # XGBoost
        xgb_results = evaluate_model(
            xgb_model,
            domain_data[test_site]['full_X'],
            domain_data[test_site]['full_y']
        )
        out_domain_results['XGBoost'][train_site][test_site] = xgb_results
        
        # TabICL
        tabicl_results = evaluate_model(
            tabicl_model,
            domain_data[test_site]['full_X'],
            domain_data[test_site]['full_y']
        )
        out_domain_results['TabICL'][train_site][test_site] = tabicl_results
        
        print(f"  {train_site} → {test_site}:")
        print(f"    XGBoost: {xgb_results['accuracy']:.4f}")
        print(f"    TabICL:  {tabicl_results['accuracy']:.4f}")

## 🗺️ Step 8: Domain Transfer Matrix Visualization

In [ ]:
# Create transfer matrices
sites_list = list(domain_data.keys())
n_sites = len(sites_list)

xgb_matrix = np.zeros((n_sites, n_sites))
tabicl_matrix = np.zeros((n_sites, n_sites))

for i, train_site in enumerate(sites_list):
    for j, test_site in enumerate(sites_list):
        if train_site == test_site:
            # Use in-domain results for diagonal
            xgb_matrix[i, j] = in_domain_results['XGBoost'][train_site]['accuracy']
            tabicl_matrix[i, j] = in_domain_results['TabICL'][train_site]['accuracy']
        else:
            xgb_matrix[i, j] = out_domain_results['XGBoost'][train_site][test_site]['accuracy']
            tabicl_matrix[i, j] = out_domain_results['TabICL'][train_site][test_site]['accuracy']

# Visualize matrices
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 5))

# XGBoost matrix
sns.heatmap(xgb_matrix, annot=True, fmt='.3f',
            xticklabels=sites_list, yticklabels=sites_list,
            cmap='RdYlGn', vmin=0, vmax=1,
            cbar_kws={'label': 'Accuracy'},
            linewidths=0.5, linecolor='gray', ax=ax1)
ax1.set_title('XGBoost Domain Transfer', fontsize=12, fontweight='bold')
ax1.set_ylabel('Train Site')
ax1.set_xlabel('Test Site')

# TabICL matrix
sns.heatmap(tabicl_matrix, annot=True, fmt='.3f',
            xticklabels=sites_list, yticklabels=sites_list,
            cmap='RdYlGn', vmin=0, vmax=1,
            cbar_kws={'label': 'Accuracy'},
            linewidths=0.5, linecolor='gray', ax=ax2)
ax2.set_title('TabICL Domain Transfer', fontsize=12, fontweight='bold')
ax2.set_ylabel('Train Site')
ax2.set_xlabel('Test Site')

# Difference matrix (TabICL - XGBoost)
diff_matrix = tabicl_matrix - xgb_matrix
sns.heatmap(diff_matrix, annot=True, fmt='.3f',
            xticklabels=sites_list, yticklabels=sites_list,
            cmap='coolwarm', center=0, vmin=-0.2, vmax=0.2,
            cbar_kws={'label': 'Difference (TabICL - XGBoost)'},
            linewidths=0.5, linecolor='gray', ax=ax3)
ax3.set_title('Performance Difference', fontsize=12, fontweight='bold')
ax3.set_ylabel('Train Site')
ax3.set_xlabel('Test Site')

# Highlight diagonal
for ax in [ax1, ax2, ax3]:
    for i in range(n_sites):
        ax.add_patch(plt.Rectangle((i, i), 1, 1, fill=False, edgecolor='blue', lw=2))

plt.tight_layout()
plt.show()

print("\n💡 Blue boxes indicate in-domain performance (diagonal)")
print("💡 Red in difference matrix: XGBoost better | Blue: TabICL better")

## 🌐 Step 9: Cross-Domain Performance

In [ ]:
# Cross-domain performance evaluation
print("🌐 Evaluating Cross-Domain Performance...")
print("(Train on multiple sites, test on held-out)")
print("="*60)

cross_domain_results = {'XGBoost': {}, 'TabICL': {}}

for held_out_site in domain_data.keys():
    # Combine all other sites for training
    train_sites = [s for s in domain_data.keys() if s != held_out_site]
    
    X_train_list = [domain_data[s]['full_X'] for s in train_sites]
    y_train_list = [domain_data[s]['full_y'] for s in train_sites]
    
    X_train_combined = pd.concat(X_train_list, axis=0)
    y_train_combined = pd.concat(y_train_list, axis=0)
    
    print(f"\n📍 Training on {', '.join(train_sites)}")
    print(f"   Testing on {held_out_site}")
    print(f"   Combined training size: {len(X_train_combined)} samples")
    
    # Train XGBoost
    xgb_model = XGBoostWrapper(verbose=False)
    xgb_model.fit(X_train_combined, y_train_combined)
    xgb_results = evaluate_model(
        xgb_model,
        domain_data[held_out_site]['full_X'],
        domain_data[held_out_site]['full_y']
    )
    cross_domain_results['XGBoost'][held_out_site] = xgb_results
    
    # Train TabICL
    tabicl_model = TabICLWrapper(n_context_samples=150, verbose=False)
    tabicl_model.fit(X_train_combined, y_train_combined)
    tabicl_results = evaluate_model(
        tabicl_model,
        domain_data[held_out_site]['full_X'],
        domain_data[held_out_site]['full_y']
    )
    cross_domain_results['TabICL'][held_out_site] = tabicl_results
    
    print(f"   XGBoost: {xgb_results['accuracy']:.4f}")
    print(f"   TabICL:  {tabicl_results['accuracy']:.4f}")

## 📈 Step 10: Comprehensive Performance Comparison

In [ ]:
# Comprehensive comparison
performance_summary = []

for site in sites_list:
    # In-domain
    performance_summary.append({
        'Site': site,
        'Scenario': 'In-Domain',
        'Model': 'XGBoost',
        'Accuracy': in_domain_results['XGBoost'][site]['accuracy']
    })
    performance_summary.append({
        'Site': site,
        'Scenario': 'In-Domain',
        'Model': 'TabICL',
        'Accuracy': in_domain_results['TabICL'][site]['accuracy']
    })
    
    # Out-domain average
    xgb_out_accs = [out_domain_results['XGBoost'][train][site]['accuracy']
                   for train in sites_list if train != site]
    tabicl_out_accs = [out_domain_results['TabICL'][train][site]['accuracy']
                      for train in sites_list if train != site]
    
    performance_summary.append({
        'Site': site,
        'Scenario': 'Out-Domain',
        'Model': 'XGBoost',
        'Accuracy': np.mean(xgb_out_accs) if xgb_out_accs else 0
    })
    performance_summary.append({
        'Site': site,
        'Scenario': 'Out-Domain',
        'Model': 'TabICL',
        'Accuracy': np.mean(tabicl_out_accs) if tabicl_out_accs else 0
    })
    
    # Cross-domain
    performance_summary.append({
        'Site': site,
        'Scenario': 'Cross-Domain',
        'Model': 'XGBoost',
        'Accuracy': cross_domain_results['XGBoost'][site]['accuracy']
    })
    performance_summary.append({
        'Site': site,
        'Scenario': 'Cross-Domain',
        'Model': 'TabICL',
        'Accuracy': cross_domain_results['TabICL'][site]['accuracy']
    })

df_perf = pd.DataFrame(performance_summary)

# Create interactive plot
fig = px.bar(df_perf, x='Site', y='Accuracy', color='Model',
             facet_col='Scenario',
             title='Model Performance Comparison: XGBoost vs TabICL',
             barmode='group', height=400,
             color_discrete_map={'XGBoost': '#2E7D32', 'TabICL': '#1976D2'})

fig.update_layout(
    xaxis_tickangle=-45,
    yaxis_range=[0, 1],
    font=dict(size=11)
)

fig.show()

# Summary statistics
print("\n📊 Performance Summary by Scenario:")
print("="*60)

summary_stats = df_perf.groupby(['Scenario', 'Model'])['Accuracy'].agg(['mean', 'std']).round(4)
print(summary_stats)

## 📊 Step 11: Statistical Significance Testing

In [ ]:
# Statistical significance testing
print("📊 Statistical Significance Testing")
print("="*60)

# Collect paired accuracies for each scenario
scenarios = ['In-Domain', 'Out-Domain', 'Cross-Domain']

for scenario in scenarios:
    print(f"\n{scenario} Performance:")
    print("-"*40)
    
    if scenario == 'In-Domain':
        xgb_accs = [in_domain_results['XGBoost'][s]['accuracy'] for s in sites_list]
        tabicl_accs = [in_domain_results['TabICL'][s]['accuracy'] for s in sites_list]
    elif scenario == 'Out-Domain':
        xgb_accs = []
        tabicl_accs = []
        for test_site in sites_list:
            xgb_site = [out_domain_results['XGBoost'][train][test_site]['accuracy']
                       for train in sites_list if train != test_site]
            tabicl_site = [out_domain_results['TabICL'][train][test_site]['accuracy']
                          for train in sites_list if train != test_site]
            xgb_accs.extend(xgb_site)
            tabicl_accs.extend(tabicl_site)
    else:  # Cross-Domain
        xgb_accs = [cross_domain_results['XGBoost'][s]['accuracy'] for s in sites_list]
        tabicl_accs = [cross_domain_results['TabICL'][s]['accuracy'] for s in sites_list]
    
    # Paired t-test
    if len(xgb_accs) > 1:
        t_stat, p_value = stats.ttest_rel(tabicl_accs, xgb_accs)
        
        mean_xgb = np.mean(xgb_accs)
        mean_tabicl = np.mean(tabicl_accs)
        diff = mean_tabicl - mean_xgb
        
        print(f"  XGBoost mean:  {mean_xgb:.4f} (±{np.std(xgb_accs):.4f})")
        print(f"  TabICL mean:   {mean_tabicl:.4f} (±{np.std(tabicl_accs):.4f})")
        print(f"  Difference:    {diff:+.4f}")
        print(f"  t-statistic:   {t_stat:.4f}")
        print(f"  p-value:       {p_value:.4f}")
        
        if p_value < 0.05:
            better = "TabICL" if diff > 0 else "XGBoost"
            print(f"  ✅ {better} is significantly better (p < 0.05)")
        else:
            print(f"  ⚪ No significant difference (p >= 0.05)")

# Wilcoxon signed-rank test (non-parametric alternative)
print("\n" + "="*60)
print("Wilcoxon Signed-Rank Test (Non-parametric):")
print("-"*40)

all_xgb = []
all_tabicl = []

for site in sites_list:
    all_xgb.append(in_domain_results['XGBoost'][site]['accuracy'])
    all_tabicl.append(in_domain_results['TabICL'][site]['accuracy'])
    all_xgb.append(cross_domain_results['XGBoost'][site]['accuracy'])
    all_tabicl.append(cross_domain_results['TabICL'][site]['accuracy'])

if len(all_xgb) > 1:
    statistic, p_value = stats.wilcoxon(all_tabicl, all_xgb)
    print(f"Overall comparison (all scenarios):")
    print(f"  Wilcoxon statistic: {statistic:.4f}")
    print(f"  p-value: {p_value:.4f}")
    
    if p_value < 0.05:
        median_diff = np.median(np.array(all_tabicl) - np.array(all_xgb))
        better = "TabICL" if median_diff > 0 else "XGBoost"
        print(f"  ✅ {better} performs significantly better overall")
    else:
        print(f"  ⚪ No significant overall difference")

## 📉 Step 12: Domain Shift Analysis

In [ ]:
# Domain shift analysis
domain_shift_analysis = []

for site in sites_list:
    # XGBoost
    xgb_in = in_domain_results['XGBoost'][site]['accuracy']
    xgb_cross = cross_domain_results['XGBoost'][site]['accuracy']
    xgb_out = np.mean([out_domain_results['XGBoost'][train][site]['accuracy']
                      for train in sites_list if train != site])
    
    # TabICL
    tabicl_in = in_domain_results['TabICL'][site]['accuracy']
    tabicl_cross = cross_domain_results['TabICL'][site]['accuracy']
    tabicl_out = np.mean([out_domain_results['TabICL'][train][site]['accuracy']
                         for train in sites_list if train != site])
    
    domain_shift_analysis.append({
        'Site': site,
        'XGBoost_In': xgb_in,
        'XGBoost_Cross': xgb_cross,
        'XGBoost_Out': xgb_out,
        'XGBoost_Drop': xgb_in - xgb_cross,
        'TabICL_In': tabicl_in,
        'TabICL_Cross': tabicl_cross,
        'TabICL_Out': tabicl_out,
        'TabICL_Drop': tabicl_in - tabicl_cross,
        'Robustness_Diff': (tabicl_in - tabicl_cross) - (xgb_in - xgb_cross)
    })

df_shift = pd.DataFrame(domain_shift_analysis)

# Visualize domain shift
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Performance drop comparison
x = np.arange(len(sites_list))
width = 0.35

xgb_drops = df_shift['XGBoost_Drop'].values
tabicl_drops = df_shift['TabICL_Drop'].values

axes[0].bar(x - width/2, xgb_drops, width, label='XGBoost', color='#FF6B6B', alpha=0.7)
axes[0].bar(x + width/2, tabicl_drops, width, label='TabICL', color='#4ECDC4', alpha=0.7)
axes[0].set_xlabel('Site', fontsize=12)
axes[0].set_ylabel('Accuracy Drop (In→Cross)', fontsize=12)
axes[0].set_title('Domain Shift Impact', fontsize=14, fontweight='bold')
axes[0].set_xticks(x)
axes[0].set_xticklabels(sites_list)
axes[0].legend()
axes[0].grid(True, alpha=0.3)
axes[0].axhline(y=0, color='black', linestyle='-', linewidth=0.5)

# Absolute performance across scenarios
scenarios_x = ['In-Domain', 'Cross-Domain', 'Out-Domain']
xgb_means = [df_shift['XGBoost_In'].mean(), 
             df_shift['XGBoost_Cross'].mean(),
             df_shift['XGBoost_Out'].mean()]
tabicl_means = [df_shift['TabICL_In'].mean(),
                df_shift['TabICL_Cross'].mean(),
                df_shift['TabICL_Out'].mean()]

axes[1].plot(scenarios_x, xgb_means, 'o-', label='XGBoost',
            linewidth=2, markersize=8, color='#2E7D32')
axes[1].plot(scenarios_x, tabicl_means, 's-', label='TabICL',
            linewidth=2, markersize=8, color='#1976D2')
axes[1].set_ylabel('Average Accuracy', fontsize=12)
axes[1].set_title('Performance Across Scenarios', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)
axes[1].set_ylim([0, 1])

plt.tight_layout()
plt.show()

print("\n📊 Domain Robustness Analysis:")
print("="*60)
print(df_shift[['Site', 'XGBoost_Drop', 'TabICL_Drop', 'Robustness_Diff']].round(4))
print("\n💡 Negative Robustness_Diff means TabICL is more robust to domain shift")

## 📝 Step 13: Final Report and Conclusions

In [ ]:
# Generate final report
print("="*80)
print(" "*15 + "TABICL VS XGBOOST: FINAL COMPARISON REPORT")
print("="*80)

# Dataset overview
print(f"\n📊 DATASET OVERVIEW:")
print(f"  • Total samples: {len(df):,}")
print(f"  • Number of features: {len(df.columns) - 2}")
print(f"  • Number of classes: {df['va34'].nunique()}")
print(f"  • Number of sites: {len(sites_list)}")
print(f"  • TabICL feature reduction: {len(df.columns) - 2} → 100 features")

# Performance summary
print(f"\n📈 OVERALL PERFORMANCE SUMMARY:")

scenarios_perf = {}
for scenario in ['In-Domain', 'Out-Domain', 'Cross-Domain']:
    xgb_mean = df_perf[(df_perf['Scenario'] == scenario) & 
                       (df_perf['Model'] == 'XGBoost')]['Accuracy'].mean()
    tabicl_mean = df_perf[(df_perf['Scenario'] == scenario) & 
                          (df_perf['Model'] == 'TabICL')]['Accuracy'].mean()
    scenarios_perf[scenario] = {'XGBoost': xgb_mean, 'TabICL': tabicl_mean}
    
    print(f"\n  {scenario}:")
    print(f"    • XGBoost: {xgb_mean:.4f}")
    print(f"    • TabICL:  {tabicl_mean:.4f}")
    diff = tabicl_mean - xgb_mean
    if abs(diff) > 0.01:
        winner = "TabICL" if diff > 0 else "XGBoost"
        print(f"    • Winner:  {winner} (+{abs(diff):.4f})")
    else:
        print(f"    • Result:  Comparable performance")

# Domain robustness
print(f"\n📉 DOMAIN ROBUSTNESS:")
avg_xgb_drop = df_shift['XGBoost_Drop'].mean()
avg_tabicl_drop = df_shift['TabICL_Drop'].mean()
print(f"  • Average XGBoost accuracy drop (In→Cross): {avg_xgb_drop:.4f}")
print(f"  • Average TabICL accuracy drop (In→Cross):  {avg_tabicl_drop:.4f}")
if abs(avg_tabicl_drop) < abs(avg_xgb_drop):
    print(f"  • ✅ TabICL is more robust to domain shift")
else:
    print(f"  • ✅ XGBoost is more robust to domain shift")

# Training efficiency
print(f"\n⏱️ TRAINING EFFICIENCY:")
avg_xgb_time = np.mean(list(training_times['XGBoost'].values()))
avg_tabicl_time = np.mean(list(training_times['TabICL'].values()))
print(f"  • Average XGBoost training time: {avg_xgb_time:.2f}s")
print(f"  • Average TabICL training time:  {avg_tabicl_time:.2f}s")
print(f"  • Speed ratio: {avg_xgb_time/avg_tabicl_time:.2f}x")

# Best and worst cases
print(f"\n🏆 BEST & WORST PERFORMERS:")

# Find best performances
xgb_best_site = max(sites_list, key=lambda x: in_domain_results['XGBoost'][x]['accuracy'])
xgb_best_acc = in_domain_results['XGBoost'][xgb_best_site]['accuracy']
tabicl_best_site = max(sites_list, key=lambda x: in_domain_results['TabICL'][x]['accuracy'])
tabicl_best_acc = in_domain_results['TabICL'][tabicl_best_site]['accuracy']

print(f"  Best in-domain:")
print(f"    • XGBoost: {xgb_best_site} ({xgb_best_acc:.4f})")
print(f"    • TabICL:  {tabicl_best_site} ({tabicl_best_acc:.4f})")

# Key insights
print(f"\n💡 KEY INSIGHTS:")

insights = []

# Compare overall performance
overall_xgb = np.mean([scenarios_perf[s]['XGBoost'] for s in scenarios_perf])
overall_tabicl = np.mean([scenarios_perf[s]['TabICL'] for s in scenarios_perf])

if overall_tabicl > overall_xgb:
    insights.append(f"TabICL achieves {(overall_tabicl - overall_xgb):.4f} higher average accuracy")
else:
    insights.append(f"XGBoost achieves {(overall_xgb - overall_tabicl):.4f} higher average accuracy")

if abs(avg_tabicl_drop) < abs(avg_xgb_drop):
    insights.append(f"TabICL shows {abs(avg_xgb_drop - avg_tabicl_drop):.4f} better domain robustness")

if avg_tabicl_time < avg_xgb_time:
    insights.append(f"TabICL trains {avg_xgb_time/avg_tabicl_time:.1f}x faster")
else:
    insights.append(f"XGBoost trains {avg_tabicl_time/avg_xgb_time:.1f}x faster")

insights.append("TabICL requires no hyperparameter tuning")
insights.append("TabICL uses in-context learning without gradient updates")

for i, insight in enumerate(insights, 1):
    print(f"  {i}. {insight}")

# Recommendations
print(f"\n🎯 RECOMMENDATIONS:")
print(f"  • Use TabICL when:")
print(f"    - Domain shift is a concern")
print(f"    - Limited hyperparameter tuning resources")
print(f"    - Need quick prototyping")
print(f"    - Working with few-shot scenarios")
print(f"\n  • Use XGBoost when:")
print(f"    - Maximum accuracy is critical")
print(f"    - Single domain deployment")
print(f"    - Interpretability is important")
print(f"    - Computational resources are limited")

print("\n" + "="*80)
print("✅ Comparison Analysis Complete!")
print("="*80)

## 💾 Optional: Save Results to Drive

In [ ]:
# Optional: Save results
save_results = input("Save results to CSV? (y/n): ").lower() == 'y'

if save_results:
    import os
    os.makedirs('comparison_results', exist_ok=True)
    
    # Save performance summary
    df_perf.to_csv('comparison_results/model_performance.csv', index=False)
    df_shift.to_csv('comparison_results/domain_shift_analysis.csv', index=False)
    
    # Save transfer matrices
    pd.DataFrame(xgb_matrix, index=sites_list, columns=sites_list).to_csv(
        'comparison_results/xgboost_transfer_matrix.csv'
    )
    pd.DataFrame(tabicl_matrix, index=sites_list, columns=sites_list).to_csv(
        'comparison_results/tabicl_transfer_matrix.csv'
    )
    
    # Save summary statistics
    with open('comparison_results/summary_report.txt', 'w') as f:
        f.write("TabICL vs XGBoost Comparison Summary\n")
        f.write("="*50 + "\n\n")
        f.write(f"Overall XGBoost accuracy: {overall_xgb:.4f}\n")
        f.write(f"Overall TabICL accuracy: {overall_tabicl:.4f}\n")
        f.write(f"Domain robustness - XGBoost drop: {avg_xgb_drop:.4f}\n")
        f.write(f"Domain robustness - TabICL drop: {avg_tabicl_drop:.4f}\n")
    
    print("✅ Results saved to 'comparison_results/' directory")
    
    # Mount Google Drive if in Colab
    try:
        from google.colab import drive
        drive.mount('/content/drive')
        !cp -r comparison_results /content/drive/MyDrive/
        print("✅ Results also copied to Google Drive")
    except:
        print("📁 Results saved locally")
else:
    print("Results not saved.")